In [ ]:
# Prophet Account Receivable Forecasting Demo
# MS Fabric Lakehouse Notebook

# Install Prophet (run this cell first)
%pip install prophet

import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# =============================================================================
# 1. CREATE SAMPLE AR DATA
# =============================================================================

# Generate sample AR data - replace this section with your actual data loading
np.random.seed(43)
start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Create realistic AR patterns
sample_data = []
for date in date_range:
    base_amount = 50000  # Base daily collections
    
    # Add seasonality patterns
    month_effect = 1.2 if date.month in [12, 1, 3, 6] else 1.0  # Quarter-end spikes
    day_effect = 0.7 if date.weekday() >= 5 else 1.0  # Lower weekend collections
    
    # Add trend
    years_since_start = (date - start_date).days / 365.25
    trend_effect = 1 + (years_since_start * 0.05)  # 5% annual growth
    
    # Add some randomness
    noise = np.random.normal(1, 0.15)
    
    daily_collections = base_amount * month_effect * day_effect * trend_effect * noise
    daily_collections = max(daily_collections, 0)  # No negative collections
    
    sample_data.append({
        'date': date,
        'collections': daily_collections,
        'customer_segment': np.random.choice(['Enterprise', 'SMB', 'Government'], p=[0.5, 0.3, 0.2])
    })

df_raw = pd.DataFrame(sample_data)

# Aggregate to monthly data
df_monthly = df_raw.groupby([df_raw['date'].dt.to_period('M')]).agg({
    'collections': 'sum'
}).reset_index()
df_monthly['date'] = df_monthly['date'].dt.to_timestamp(how='end') #transform to month-end dates

print("Sample AR Data (Monthly Collections):")
print(df_monthly.head(10))
print(f"\nData shape: {df_monthly.shape}")
print(f"Date range: {df_monthly['date'].min()} to {df_monthly['date'].max()}")

In [ ]:
# =============================================================================
# 1.5 WRITE SAMPLE AR DATA TO LAKEHOUSE
# =============================================================================

# Convert to Spark DataFrame
df_monthly_spark = spark.createDataFrame(df_monthly)

# Write to Delta table in your Lakehouse
df_monthly_spark.write.format("delta").mode("overwrite").saveAsTable("ar_collections_monthly")

print("✓ Data uploaded to Lakehouse table: ar_collections_monthly")

In [ ]:
# =============================================================================
# 2. LOAD AR DATA FROM LAKEHOUSE
# =============================================================================

"""
# Load from Lakehouse table
df_ar = spark.sql('''
    SELECT 
        date as date,
        SUM(collections) as collections
    FROM Prophet_Lakehouse.dbo.ar_collections_monthly
    WHERE date >= '2020-01-01'
    GROUP BY date
    ORDER BY date
''').toPandas()
"""

# Load from Delta table
df_ar = spark.read.format("delta").load("Tables/dbo/ar_collections_monthly").toPandas()

# Count rows (use len() for pandas DataFrame)
row_count = len(df_ar)

print(f"✓ Data loaded: {row_count} rows")

In [ ]:
# =============================================================================
# 3. PREPARE DATA FOR PROPHET
# =============================================================================

# Prophet requires specific column names: 'ds' (datestamp) and 'y' (value)
#prophet_data = df_ar[['date', 'collections']].copy()
#prophet_data.columns = ['ds', 'y']


df0 = df_ar.copy()
ds = (pd.to_datetime(df0['date'], utc=True)
        .dt.tz_localize(None)
        .astype('datetime64[ns]'))

prophet_data = (pd.DataFrame({'ds': ds, 'y': df0['collections']})
                  .set_index('ds')
                  .resample('M')['y'].sum()
                  .rename_axis('ds').reset_index()
                  .sort_values('ds').reset_index(drop=True))

# Asserts
assert pd.infer_freq(prophet_data['ds']) == 'M'
assert prophet_data['ds'].dt.is_month_end.all()

# Remove any missing values
prophet_data = prophet_data.dropna()

print("\nData prepared for Prophet:")
print(prophet_data.head())

In [ ]:
# =============================================================================
# 4. INITIALIZE PROPHET MODEL
# =============================================================================

# Initialize Prophet model
model = Prophet(
    # ==================== TREND PARAMETERS ====================
    growth='linear',  # 'linear' or 'logistic'
                      # linear: Unlimited growth (good for most business metrics)
                      # logistic: Growth with saturation limit (set cap/floor)
                      # TUNE: If actuals show clear ceiling/floor, switch to logistic
    
    changepoint_prior_scale=0.05,  # Range: 0.001 to 0.5+ (default: 0.05)
                                   # Controls trend flexibility/changepoint sensitivity
                                   # Lower = smoother, fewer trend changes
                                   # Higher = more reactive to shifts
                                   # TUNE BY AR: 
                                   #   - Forecast misses level shifts → INCREASE (try 0.1, 0.5, 1.0)
                                   #   - Forecast too jagged/overfits → DECREASE (try 0.01, 0.001)
                                   #   - Systematic over/under at trend breaks → INCREASE
    
    n_changepoints=25,  # Number of potential changepoints (default: 25)
                        # More points = can detect more trend shifts
                        # TUNE BY AR:
                        #   - Multiple missed level changes → INCREASE to 50-100
                        #   - Overfitting with erratic changes → DECREASE to 10-15
    
    changepoint_range=0.95,  # Fraction of history where changepoints allowed (default: 0.8)
                             # 0.8 = only first 80% of training data
                             # 0.95 = allows trend changes closer to forecast period
                             # TUNE BY AR:
                             #   - Forecast misses recent shift → INCREASE to 0.9-0.95
                             #   - Helps when level changed right before forecast period
    
    # ==================== SEASONALITY PARAMETERS ====================
    yearly_seasonality=True,  # Can also be integer (Fourier terms)
                              # True = auto (10 terms), False = none
                              # Integer (e.g., 10, 20) = custom complexity
                              # TUNE BY AR:
                              #   - Seasonal pattern too smooth → INCREASE to 15-20
                              #   - Seasonal pattern too wiggly → DECREASE to 5-8
    
    weekly_seasonality=False,  # Same logic as yearly
                               # Only relevant if data is daily/hourly
                               # True = 3 Fourier terms
    
    daily_seasonality=False,   # Only for hourly/minute data
                               # True = 4 Fourier terms
    
    seasonality_prior_scale=10.0,  # Range: 0.01 to 100+ (default: 10.0)
                                    # Controls seasonality strength/amplitude
                                    # Lower = weaker seasonal effect
                                    # Higher = stronger seasonal peaks/troughs
                                    # TUNE BY AR:
                                    #   - Forecast peaks too flat → INCREASE (try 15, 20, 30)
                                    #   - Forecast peaks too extreme → DECREASE (try 5, 1, 0.1)
                                    #   - Check residuals: if seasonal pattern remains → INCREASE
    
    seasonality_mode='additive',  # 'additive' or 'multiplicative'
                                   # additive: Seasonal variation stays constant (+/- same amount)
                                   # multiplicative: Variation grows with level (percentage-based)
                                   # TUNE BY AR:
                                   #   - Seasonal swings grow over time → use 'multiplicative'
                                   #   - Seasonal swings constant → use 'additive'
                                   #   - Visual check: does % variation stay similar? → multiplicative
    
    # ==================== UNCERTAINTY PARAMETERS ====================
    interval_width=0.95,  # Confidence interval width (default: 0.80)
                          # 0.80 = 80% interval, 0.95 = 95% interval
                          # Wider = more conservative bounds
                          # TUNE BY AR:
                          #   - Too many actuals outside bands → INCREASE to 0.95
                          #   - Bands too wide to be useful → DECREASE to 0.60-0.70
                          #   - This doesn't affect forecast, only uncertainty bands
    
    mcmc_samples=300,  # Bayesian sampling for uncertainty (default: 0 = no MCMC)
                     # 0 = fast MAP estimation
                     # 300+ = slower but better uncertainty estimates
                     # TUNE: Use 300-500 if you need reliable prediction intervals
                     #       (doesn't change point forecast, only affects uncertainty)
    
    # ==================== OTHER USEFUL PARAMETERS ====================
    
    # holidays=holidays_df,  # DataFrame with holiday dates and effects
                              # Columns: ['ds', 'holiday', 'lower_window', 'upper_window']
                              # TUNE BY AR:
                              #   - Spikes on specific dates → add those dates as holidays
                              #   - Check residuals for recurring anomalies
    
    # Add custom seasonalities for non-standard periods:
    # model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    # model.add_seasonality(name='quarterly', period=91.25, fourier_order=8)
    # TUNE BY AR: If you see patterns at specific frequencies
    
    # Add regressors (external variables):
    # model.add_regressor('promo', prior_scale=0.5, mode='additive')
    # TUNE BY AR: If actuals correlate with external events/variables
)

# ==================== ADVANCED TUNING BASED ON AR ANALYSIS ====================

# After fitting, analyze residuals:
# residuals = actual - forecast['yhat']
# 
# PATTERN → ACTION:
# 
# 1. Systematic over/under prediction:
#    - Early period: Increase changepoint_range
#    - Recent period: Check if training data includes recent level
#    - Throughout: Increase changepoint_prior_scale
#
# 2. Missed seasonal peaks:
#    - Increase seasonality_prior_scale
#    - Try seasonality_mode='multiplicative'
#    - Increase fourier_order for yearly_seasonality
#
# 3. Forecast too smooth:
#    - Increase changepoint_prior_scale (0.1 → 0.5 → 1.0)
#    - Increase seasonality_prior_scale (10 → 20 → 30)
#
# 4. Forecast too volatile/overfit:
#    - Decrease changepoint_prior_scale (0.05 → 0.01)
#    - Decrease seasonality_prior_scale (10 → 5 → 1)
#    - Reduce n_changepoints

"""
# Add custom holidays/events (e.g., fiscal year end, major customer payments)
holidays = pd.DataFrame({
    'holiday': 'fiscal_year_end',
    'ds': pd.to_datetime(['2021-12-31', '2022-12-31', '2023-12-31', '2024-12-31', '2025-12-31']),
    'lower_window': -5,
    'upper_window': 5,
})
"""

"""
# Add external regressors (if you have economic indicators, etc.)

# Example: Adding GDP growth as regressor
prophet_data['gdp_growth'] = your_gdp_data  # Add your external data
future_advanced['gdp_growth'] = future_gdp_data  # Add future values

model_with_regressors = Prophet()
model_with_regressors.add_regressor('gdp_growth')
model_with_regressors.fit(prophet_data)
"""


# Fit the model
print("\nFitting Prophet model...")
model.fit(prophet_data)

In [ ]:
# =============================================================================
# 5. CREATE FUTURE PREDICTIONS
# =============================================================================

# Create future dataframe for next 12 months
future = model.make_future_dataframe(periods=12, freq='M')
print(f"\nPredicting {12} months ahead...")

# Make predictions
forecast = model.predict(future)

# Display forecast results
forecast_summary = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(12)
forecast_summary.columns = ['Date', 'Predicted_Collections', 'Lower_Bound', 'Upper_Bound']
print("\nNext 12 months forecast:")
print(forecast_summary)

In [ ]:
# =============================================================================
# 6. VISUALIZATIONS
# =============================================================================

# Plot forecast
fig1 = model.plot(forecast, figsize=(12, 6))
plt.title('Account Receivable Collections Forecast')
plt.xlabel('Date')
plt.ylabel('Collections ($)')
plt.show()

# Plot components (trend, seasonality)
fig2 = model.plot_components(forecast, figsize=(12, 8))
plt.show()

In [ ]:
# =============================================================================
# 7. SAVE RESULTS TO LAKEHOUSE
# =============================================================================

# Prepare forecast data for saving
forecast_final = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
forecast_final.columns = ['forecast_date', 'predicted_collections', 'lower_bound', 'upper_bound']
forecast_final['model_run_date'] = datetime.now()
forecast_final['model_type'] = 'Prophet'

# Convert to Spark DataFrame and save to Lakehouse
forecast_spark = spark.createDataFrame(forecast_final)

# Save to Delta table
forecast_spark.write.format("delta").mode("overwrite").saveAsTable("Prophet_Lakehouse.dbo.ar_forecast")
print("Forecast saved to Lakehouse table: Prophet_Lakehouse.dbo.ar_forecast")

print("\n" + "="*50)
print("AR FORECASTING COMPLETE")
print("="*50)

In [ ]:
# =============================================================================
# MODEL BACK-TESTING AND CROSS-VALIDATION
# =============================================================================

import numpy as np
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# 1. DEFINE PARAMETER GRID
# =============================================================================

param_grid = {
    'changepoint_prior_scale': [0.05, 0.1, 0.5],
    'seasonality_prior_scale': [10.0, 20.0, 30.0],
    'seasonality_mode': ['additive', 'multiplicative'],
    'changepoint_range': [0.9, 0.95], 
}

# Fixed parameters (not tuned)
base_params = {
    'growth': 'linear',
    'yearly_seasonality': True,
    'weekly_seasonality': False,
    'daily_seasonality': False,
    'n_changepoints': 25,
    'interval_width': 0.80,
}

# =============================================================================
# 2. CROSS-VALIDATION FUNCTION
# =============================================================================

def evaluate_params(params_dict, data, cv_config):
    """
    Evaluate a parameter set using time-series cross-validation
    
    Args:
        params_dict: Dictionary of Prophet parameters
        data: Training data (DataFrame with 'ds' and 'y')
        cv_config: Cross-validation configuration
    
    Returns:
        Dictionary with performance metrics
    """
    try:
        # Combine base params with test params
        full_params = {**base_params, **params_dict}
        
        # Train model
        model = Prophet(**full_params)
        model.fit(data)
        
        # Cross-validation
        df_cv = cross_validation(
            model,
            initial=cv_config['initial'],
            period=cv_config['period'],
            horizon=cv_config['horizon'],
            parallel="processes"  # Speed up with parallel processing
        )
        
        # Calculate metrics
        df_metrics = performance_metrics(df_cv)
        
        return {
            'params': params_dict,
            'mape': df_metrics['mape'].mean(),
            'mae': df_metrics['mae'].mean(),
            'rmse': df_metrics['rmse'].mean(),
            'coverage': df_metrics['coverage'].mean(),  # % of actuals in prediction interval
            'df_cv': df_cv,
            'df_metrics': df_metrics
        }
    
    except Exception as e:
        print(f"Error with params {params_dict}: {str(e)}")
        return {
            'params': params_dict,
            'mape': np.inf,
            'mae': np.inf,
            'rmse': np.inf,
            'coverage': 0,
            'error': str(e)
        }

# =============================================================================
# 3. GRID SEARCH WITH CROSS-VALIDATION
# =============================================================================

# Cross-validation configuration
cv_config = {
    'initial': '730 days',   # Minimum training data (2 years)
    'period': '180 days',    # Step between cutoff dates (6 months)
    'horizon': '365 days',   # Forecast horizon to evaluate (1 year)
}

print("Starting Grid Search with Cross-Validation...")
print(f"CV Config: {cv_config}")
print(f"Testing {np.prod([len(v) for v in param_grid.values()])} parameter combinations\n")

# Generate all parameter combinations
keys = param_grid.keys()
values = param_grid.values()
param_combinations = [dict(zip(keys, v)) for v in product(*values)]

# Evaluate each combination
results = []
for i, params in enumerate(param_combinations, 1):
    print(f"Testing {i}/{len(param_combinations)}: {params}")
    result = evaluate_params(params, prophet_data, cv_config)
    results.append(result)
    print(f"  MAPE: {result['mape']:.2f}%, MAE: {result['mae']:.2f}, RMSE: {result['rmse']:.2f}\n")

# =============================================================================
# 4. ANALYZE RESULTS
# =============================================================================

# Convert to DataFrame
results_df = pd.DataFrame([
    {
        **r['params'],
        'mape': r['mape'],
        'mae': r['mae'],
        'rmse': r['rmse'],
        'coverage': r['coverage']
    }
    for r in results if r['mape'] != np.inf
])

# Sort by MAPE
results_df = results_df.sort_values('mape').reset_index(drop=True)

# Display top 10 models
print("\n" + "="*80)
print("TOP 10 MODELS BY MAPE")
print("="*80)
print(results_df.head(10).to_string(index=False))
print("\n")

# Best model
best_params = results_df.iloc[0].to_dict()
print("="*80)
print("BEST MODEL PARAMETERS")
print("="*80)
for key, value in best_params.items():
    print(f"{key:30s}: {value}")
print("\n")

# =============================================================================
# 5. VISUALIZE RESULTS
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: MAPE vs changepoint_prior_scale
ax1 = axes[0, 0]
for mode in results_df['seasonality_mode'].unique():
    subset = results_df[results_df['seasonality_mode'] == mode]
    ax1.scatter(subset['changepoint_prior_scale'], subset['mape'], 
               label=mode, alpha=0.6, s=100)
ax1.set_xlabel('Changepoint Prior Scale', fontsize=12)
ax1.set_ylabel('MAPE (%)', fontsize=12)
ax1.set_xscale('log')
ax1.legend()
ax1.set_title('MAPE vs Changepoint Prior Scale', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Plot 2: MAPE vs seasonality_prior_scale
ax2 = axes[0, 1]
for mode in results_df['seasonality_mode'].unique():
    subset = results_df[results_df['seasonality_mode'] == mode]
    ax2.scatter(subset['seasonality_prior_scale'], subset['mape'], 
               label=mode, alpha=0.6, s=100)
ax2.set_xlabel('Seasonality Prior Scale', fontsize=12)
ax2.set_ylabel('MAPE (%)', fontsize=12)
ax2.set_xscale('log')
ax2.legend()
ax2.set_title('MAPE vs Seasonality Prior Scale', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Plot 3: MAPE distribution by seasonality mode
ax3 = axes[1, 0]
results_df.boxplot(column='mape', by='seasonality_mode', ax=ax3)
ax3.set_xlabel('Seasonality Mode', fontsize=12)
ax3.set_ylabel('MAPE (%)', fontsize=12)
ax3.set_title('MAPE Distribution by Seasonality Mode', fontsize=14, fontweight='bold')
plt.sca(ax3)
plt.xticks(rotation=0)

# Plot 4: Top 15 models comparison
ax4 = axes[1, 1]
top_15 = results_df.head(15)
x_pos = np.arange(len(top_15))
ax4.barh(x_pos, top_15['mape'], color='steelblue', alpha=0.7)
ax4.set_yticks(x_pos)
ax4.set_yticklabels([f"Model {i+1}" for i in range(len(top_15))], fontsize=9)
ax4.set_xlabel('MAPE (%)', fontsize=12)
ax4.set_title('Top 15 Models by MAPE', fontsize=14, fontweight='bold')
ax4.invert_yaxis()
ax4.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('prophet_tuning_results.png', dpi=300, bbox_inches='tight')
plt.show()

# =============================================================================
# 6. DETAILED CROSS-VALIDATION PLOT FOR BEST MODEL
# =============================================================================

# Get best model's CV results
# Extract only parameter columns from best_params
param_columns = list(param_grid.keys())
best_params_only = {k: best_params[k] for k in param_columns}

# Find matching result
best_result = [r for r in results if r['params'] == best_params_only][0]
df_cv_best = best_result['df_cv']

# Plot CV results
fig, ax = plt.subplots(figsize=(15, 6))

# Plot actual values
ax.scatter(df_cv_best['ds'], df_cv_best['y'], 
          color='black', s=30, label='Actual', zorder=3)

# Plot predictions
ax.scatter(df_cv_best['ds'], df_cv_best['yhat'], 
          color='steelblue', s=20, alpha=0.5, label='Predicted', zorder=2)

# Plot prediction intervals
ax.fill_between(df_cv_best['ds'], 
                df_cv_best['yhat_lower'], 
                df_cv_best['yhat_upper'],
                alpha=0.2, color='steelblue', label='80% Interval')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Collections ($)', fontsize=12)
ax.set_title('Cross-Validation Results - Best Model', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('prophet_cv_best_model.png', dpi=300, bbox_inches='tight')
plt.show()